## Constructing Lie Algebras Examples

In [1]:
from Lie_algebras import *
import numpy as np

In [2]:
A = np.array([[1, 0], [1, 0]])
B = np.array([[2, 0], [1, 1]])

In [22]:
complete_algebra([X, I, Z], 10)

[array([[0, 1],
        [1, 0]]),
 array([[1., 0.],
        [0., 1.]]),
 array([[ 1,  0],
        [ 0, -1]]),
 array([[ 0, -2],
        [ 2,  0]])]

In [9]:
find_algebra([X, I], [Z, A, B], 10)

[array([[0, 1],
        [1, 0]]),
 array([[1., 0.],
        [0., 1.]]),
 array([[ 0, -2],
        [ 2,  0]]),
 array([[ 1, -1],
        [ 1, -1]])]

In [19]:
complete_algebra([X, I, comm(X, Z), comm(X, A)], 10)

[array([[0, 1],
        [1, 0]]),
 array([[1., 0.],
        [0., 1.]]),
 array([[ 0, -2],
        [ 2,  0]]),
 array([[ 1, -1],
        [ 1, -1]])]

In [26]:
complete_algebra([X, I], 10) == [X, I]

True

#### Tests from paper